In [ ]:
import utils.data as ud
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
import tbtools.dev as tbdev
import tbtools.panda as tbpd
import tbtools.iter as tbiter
import tbtools.strings as tbstr

In [ ]:
import multiprocessing
import tbtools.func as tbf

In [ ]:
df = ud.enhanced.get('bitbushist')

## At each exit from reinspection, note how much time it took the carcass to pass through.

In [ ]:
@tbf.unpack_args_tuple
def time_worker(uid, gr):
    output = pd.Series(np.nan, index=gr.index)
    
    for i in gr.index[gr.reinspection_change == 1]:
        k = gr.reinspection_change.loc[i:].argmin()
        t = (gr.Timestamp[k]-gr.Timestamp[i]).total_seconds()
        output[k] = t
        
    return output

def time_leader(bbh):
    with multiprocessing.Pool() as p:
        res = p.map(time_worker, bbh.reset_index().groupby('uid'))
    try:
        res = pd.concat(res, axis=0).sort_index()
    except:
        print('Error while concating!')
    return res

In [ ]:
res = time_leader(df)

In [ ]:
df['ri_s'] = res.values

In [ ]:
df2 = df[(df.reinspection_change_line == 1) & (df.reinspection_change == -1)]

In [ ]:
df2.reset_index().ri_s.argmax()

In [ ]:
df3.rolling?

In [ ]:
df3 = df2['2012-09-03']

In [ ]:
import utils.features as uf
c = uf.C(line=1)
dc = uf.delta_C_window(line=1, window='1 min', mode='+')

In [ ]:
uf.delta_C_window?

In [ ]:
morning_team = df2.index[(df2.index.hour >= 5) & (df2.index.hour <= 13)]
afternoon_team = df2.index[(df2.index.hour >= 14) & (df2.index.hour <= 22)]

In [ ]:
c[morning_team].hist(bins=25)
c[afternoon_team].hist(alpha=.7, bins=25)

In [ ]:
tbpd.hist2d(dc10[df2.index], np.sqrt(df2.ri_s), sqrt=False, np_kwargs={'bins':(15,40)})


In [ ]:
tbpd.hist2d(c10[df2.index], df2.ri_s, sqrt=True, np_kwargs=np_kwargs)
sns.plt.figure()
tbpd.hist2d(c30[df2.index], df2.ri_s, sqrt=True, np_kwargs=np_kwargs)


### Plot of C and log(median seconds per carcass in reinspection) 

Counts are sqrt

In [ ]:
tbpd = tbdev.reload(tbpd)

In [ ]:
sns.plt.figure( figsize=(12,10))
tbpd.hist2d(c[df2.index], np.log(df2.ri_s), sqrt=True,
                       np_kwargs={'bins':(25,130)})

In [ ]:
np_kwargs = {'bins':(22,30), 'range':((0,7),(0,2500))}
fig, axs = sns.plt.subplots(1,2, figsize=(10,5))
axs = axs.flatten()
for ax, team in zip(axs, (morning_team, afternoon_team)):
    tbpd.hist2d(np.sqrt(c[team]), df2.ri_s[team], sqrt=True,
                ax=ax, np_kwargs=np_kwargs)

In [ ]:
from scipy.stats import boxcox

In [ ]:
df2.ri_s.hist(bins=30)

In [ ]:
pd.Series(boxcox(df2.ri_s)[0]).hist(bins=30)

In [ ]:
tbpd = tbdev.reload(tbpd)

In [ ]:
tbpd.hist2d(c[afternoon_team], df2.ri_s[afternoon_team], 
            np_kwargs={'bins':(27,30), 'range':((0,27),(0,10000))})

In [ ]:
df2.ri_s[(df2.index.hour > 5) & (df2.index.hour <= 14)].hist(bins=40)

In [ ]:
df2.ri_s[(df2.index.hour > 14) & (df2.index.hour <= 22)].hist(bins=40)

In [ ]:
sel = slice('2012-09-26', '2012-09-28')
sns.plt.figure(figsize=(15,6))
(df2[sel].rolling(100, min_periods=10).min().ri_s/60).plot(marker='.')
c[sel].resample('10 min').mean().plot()
dc[sel].resample('10 min').median().plot()